<a href="https://colab.research.google.com/github/lostapple9/Machine-Learning-1/blob/main/Natural_Language_Processing_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=tPYj3fFJGjk&list=LL&index=13&t=16948s&ab_channel=freeCodeCamp.org

https://colab.research.google.com/drive/1ysEKrw_LE2jMndo1snrZUh5w87LQsCxk#forceEdit=true&sandboxMode=true&scrollTo=01BJLcGb4ZqK

recurrent neural network (rnn)

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [8]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


In [10]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [9]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 0s 0us/step


In [3]:
# Lets look at one review
train_data[1]
len(train_data[1])

189

In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)
len(train_data[1])
train_data

array([[    0,     0,     0, ...,    19,   178,    32],
       [    0,     0,     0, ...,    16,   145,    95],
       [    0,     0,     0, ...,     7,   129,   113],
       ...,
       [    0,     0,     0, ...,     4,  3586, 22459],
       [    0,     0,     0, ...,    12,     9,    23],
       [    0,     0,     0, ...,   204,   131,     9]], dtype=int32)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.99)

Epoch 1/10
8/8 [==============================] - 19s 2s/step - loss: 0.6727 - acc: 0.6200 - val_loss: 0.6249 - val_acc: 0.7305
Epoch 2/10
8/8 [==============================] - 16s 2s/step - loss: 0.5501 - acc: 0.7880 - val_loss: 0.5753 - val_acc: 0.7069
Epoch 3/10
8/8 [==============================] - 16s 2s/step - loss: 0.4123 - acc: 0.8840 - val_loss: 0.5929 - val_acc: 0.6718
Epoch 4/10
8/8 [==============================] - 17s 2s/step - loss: 0.3552 - acc: 0.8920 - val_loss: 0.5524 - val_acc: 0.7335
Epoch 5/10
8/8 [==============================] - 16s 2s/step - loss: 0.2443 - acc: 0.9880 - val_loss: 0.5144 - val_acc: 0.7505
Epoch 6/10
8/8 [==============================] - 17s 2s/step - loss: 0.2303 - acc: 0.9720 - val_loss: 0.5076 - val_acc: 0.7514
Epoch 7/10
8/8 [==============================] - 17s 2s/step - loss: 0.1323 - acc: 0.9960 - val_loss: 0.6355 - val_acc: 0.6890
Epoch 8/10
8/8 [==============================] - 16s 2s/step - loss: 0.1082 - acc: 0.9840 - val_loss: 0

In [9]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 18s 22ms/step - loss: 0.8017 - acc: 0.6736
[0.8016678094863892, 0.6735600233078003]


In [16]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "Death is the solution to all problems. No man - no problem"
encoded = encode_text(text)
print(encoded)


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [17]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

death is the solution to all problems no man no problem


In [24]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.92673254]
[0.9035412]


##RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [9]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [12]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [15]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.7568711e-03 -3.3817876e-03 -2.7134812e-03 ...  4.9917144e-03
    2.6450437e-03  8.7628560e-04]
  [ 1.6775844e-03 -7.4807773e-03 -4.7098943e-03 ...  5.5401464e-04
   -2.4678628e-03  2.2021420e-03]
  [ 2.9961660e-03 -7.2992025e-03 -4.3764999e-03 ... -2.1611666e-03
    1.6025380e-04 -5.8533368e-04]
  ...
  [ 1.0664448e-02 -1.3610829e-02 -4.8654750e-03 ... -2.3068420e-03
    8.8103367e-03  2.2477044e-03]
  [ 5.2844402e-03 -1.4002364e-02 -6.9910279e-03 ...  3.5343955e-03
    9.7321440e-03  3.0650478e-03]
  [ 7.8202914e-03 -1.5843622e-02 -8.2678767e-03 ... -2.0088159e-04
    3.3097942e-03  4.2608688e-03]]

 [[ 1.6640723e-03 -2.5949476e-03 -1.4087398e-03 ... -4.4505945e-03
    2.1708068e-03  3.4734893e-03]
  [-7.1785680e-04 -4.5250336e-04 -8.4495485e-05 ... -1.1942316e-02
    4.6428954e-03  9.8715071e-03]
  [ 6.1919412e-04 -1.2146594e-03 -1.6433449e-03 ... -1.1676334e-02
    6.8951081e-03  5.5377702e-03]
  ...
  [ 2.4813279e-03 -1.0278642e-02  5.0750934e-03 ... -1.4443089e

In [17]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00275687 -0.00338179 -0.00271348 ...  0.00499171  0.00264504
   0.00087629]
 [ 0.00167758 -0.00748078 -0.00470989 ...  0.00055401 -0.00246786
   0.00220214]
 [ 0.00299617 -0.0072992  -0.0043765  ... -0.00216117  0.00016025
  -0.00058533]
 ...
 [ 0.01066445 -0.01361083 -0.00486547 ... -0.00230684  0.00881034
   0.0022477 ]
 [ 0.00528444 -0.01400236 -0.00699103 ...  0.0035344   0.00973214
   0.00306505]
 [ 0.00782029 -0.01584362 -0.00826788 ... -0.00020088  0.00330979
   0.00426087]], shape=(100, 65), dtype=float32)


In [18]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-2.7568711e-03 -3.3817876e-03 -2.7134812e-03  8.7873091e-04
 -5.9595628e-04 -1.5430187e-03  2.8973111e-04  1.2007543e-03
 -1.4000967e-03 -6.7071007e-03  2.6998767e-03 -6.1708898e-03
 -5.2276528e-03 -7.6413550e-04 -1.9263417e-03 -4.0617613e-03
 -1.2429869e-03 -4.1159979e-04  4.6607084e-04  5.6571735e-04
  3.7468208e-03 -4.4322102e-03  3.1385375e-03  2.2875988e-03
  3.5037003e-03  5.3156540e-03  6.5193948e-04  5.1691255e-04
 -3.8822892e-03 -2.2949374e-03 -1.7031868e-03 -5.4736855e-04
 -2.5467428e-03 -2.7296604e-03  3.3365332e-03  4.4082138e-03
 -1.2335693e-03 -2.0115785e-03  7.5519970e-03  3.4615996e-03
 -2.9252889e-03 -7.4467179e-04  1.8341150e-03 -3.0513864e-03
 -3.2524367e-05  4.5730248e-03  1.9580452e-03  5.0668018e-03
 -6.3121552e-03 -2.5585038e-03  3.6525424e-03 -3.8569700e-03
 -2.4892611e-03  3.2353003e-03 -3.9882590e-03 -1.0292626e-03
 -5.0827404e-03  2.2975215e-05 -7.9700252e-04  3.1830196e-03
  1.3720125e-03  3.2723255e-03  4.9917144e-03  2.6450437e-03
  8.762856

In [19]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"$nbesb$RdS.o,xiRdbfH3! Xk roJrgMgvG3FfozXJfa:PCgEh;HDld,T?g&- IT-k.\ndAe'3TyeP3;L3AMMq.&KfZpFF!adliJ!"

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer='adam', loss=loss)

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [23]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 31s 160ms/step - loss: 2.6820
Epoch 2/50
172/172 [==============================] - 29s 161ms/step - loss: 1.9868
Epoch 3/50
172/172 [==============================] - 29s 162ms/step - loss: 1.7259
Epoch 4/50
172/172 [==============================] - 29s 162ms/step - loss: 1.5795
Epoch 5/50
172/172 [==============================] - 29s 162ms/step - loss: 1.4904
Epoch 6/50
172/172 [==============================] - 29s 163ms/step - loss: 1.4304
Epoch 7/50
172/172 [==============================] - 29s 163ms/step - loss: 1.3850
Epoch 8/50
172/172 [==============================] - 30s 163ms/step - loss: 1.3482
Epoch 9/50
172/172 [==============================] - 29s 162ms/step - loss: 1.3170
Epoch 10/50
172/172 [==============================] - 29s 162ms/step - loss: 1.2884
Epoch 11/50
172/172 [==============================] - 29s 162ms/step - loss: 1.2622
Epoch 12/50
172/172 [==============================] - 29s 162ms/step - lo

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [25]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [27]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: obama
obamands in the morning,
And provide te my heart?
Thou plains great king, more larks all my conveyances and one Master Froth?

QUEEN ELIZABETH:
Can give the leg plants on mind, and one in Berenta, sir?

LARTIUS:

VOLUMNIA:
I have learn'd her he be the day,
If Warwick be so near as is the rest.

DUKE OF YORK:
Go, to the painte, the people
Defy me dear-like, than sticts' to hear me; like a dear
Enforced the prince agree,
And let him crapping him to his unnatural sceneall'd:
Signior Baptista, lest you be no more.

All:
Farewell.

DUKE VINCENTIO:


KING EDWARD IV:
Black, man, obeyself; immortal pate
To his condition from his horse.

PERDITA:
Sir, my gracious lord,--

KING RICHARD III:
I swear-blow'd; he that contempt I have remember'd
Shall I nor at the people.

AEdile:
In joyful to his own report,
